# Corporación Favorita Grocery Sales Forecasting
https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data

Brick-and-mortar grocery stores are always in a delicate dance with purchasing and sales forecasting. Predict a little over, and grocers are stuck with overstocked, perishable goods. Guess a little under, and popular items quickly sell out, leaving money on the table and customers fuming.

The problem becomes more complex as retailers add new locations with unique needs, new products, ever transitioning seasonal tastes, and unpredictable product marketing. Corporación Favorita, a large Ecuadorian-based grocery retailer, knows this all too well. They operate hundreds of supermarkets, with over 200,000 different products on their shelves.

Corporación Favorita has challenged the Kaggle community to build a model that more accurately forecasts product sales. They currently rely on subjective forecasting methods with very little data to back them up and very little automation to execute plans. They’re excited to see how machine learning could better ensure they please customers by having just enough of the right products at the right time.

Despendent variable is - How many units of each kind of product were sold in each store on each day during the two-week period

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import gc

In [1]:
PATH = "data/"
!ls {PATH}

test.csv  train.csv


In [5]:
!head -5 data/train.csv
!tail -3 data/train.csv

id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,
1,2013-01-01,25,105574,1.0,
2,2013-01-01,25,105575,2.0,
3,2013-01-01,25,108079,1.0,
125497037,2017-08-15,54,2110456,192.0,False
125497038,2017-08-15,54,2113914,198.0,True
125497039,2017-08-15,54,2116416,2.0,False


In [8]:
types={'id':'int64',
       'store_nbr':'int8', 
       'item_nbr':'int32',
       'unit_sales':'float32',
       'onpromotion':'object'}

In [9]:
%%time
df_all = pd.read_csv(f'{PATH}train.csv', parse_dates = ['date'], dtype = types, infer_datetime_format = True)

CPU times: user 1min 33s, sys: 4.61 s, total: 1min 38s
Wall time: 1min 38s


In [10]:
df_all.onpromotion.fillna(False, inplace = True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True' : True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

In [11]:
%time df_all.describe(include='all')

CPU times: user 26.2 s, sys: 1.23 s, total: 27.5 s
Wall time: 27.5 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,125497040
unique,NaN,1684,NaN,NaN,NaN,2
top,NaN,2017-07-01 00:00:00,NaN,NaN,NaN,False
freq,NaN,118194,NaN,NaN,NaN,96028767
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2017-08-15 00:00:00,NaN,NaN,NaN,NaN
mean,6.274852e+07,NaN,2.746458e+01,9.727692e+05,8.554856e+00,NaN
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,2.360515e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,3.137426e+07,NaN,1.200000e+01,5.223830e+05,2.000000e+00,NaN


In [13]:
df_all.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
125497035,125497035,2017-08-15,54,2089339,4.0,False
125497036,125497036,2017-08-15,54,2106464,1.0,True
125497037,125497037,2017-08-15,54,2110456,192.0,False
125497038,125497038,2017-08-15,54,2113914,198.0,True
125497039,125497039,2017-08-15,54,2116416,2.0,False


In [11]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [12]:
df_all.to_feather('tmp/raw')

In [ ]:
df_all = 1 #pd.read_feather('tmp/raw')

Just reading date column to avoid memory error

In [7]:
df_date = df = pd.read_csv(f'{PATH}train.csv', parse_dates = ['date'],skipinitialspace=True, infer_datetime_format = True, usecols=['date'])

In [9]:
df_date.to_feather('tmp/raw_date')

In [12]:
df_date

,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
1,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
2,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
3,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
4,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
5,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
6,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
7,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
8,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
9,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400


In [10]:
%time add_datepart(df_date, 'date')

CPU times: user 46.5 s, sys: 14.2 s, total: 1min
Wall time: 2min 52s


In [11]:
df_date.to_feather('tmp/raw_date_after')

In [20]:
gc.collect()

0

In [13]:
df_all = pd.read_feather('tmp/raw')

In [14]:
df_all = pd.concat([df_all, df_date], axis=1)

Finally save our work on disk 

In [15]:
df_all.to_feather('tmp/raw_after_merge')

Next time we will read dataframe from here 
It work like check point :D

In [3]:
df_all = pd.read_feather('tmp/raw_after_merge')

In [6]:
df_all = df_all.drop(columns=['date'])

In [7]:
df_all.to_feather('tmp/raw_after_merge_drop_date')

In [3]:
df_all = pd.read_feather('tmp/raw_after_merge_drop_date')

In [4]:
df_all

,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,25,103665,2.079442,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
1,1,25,105574,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
2,2,25,105575,1.098612,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
3,3,25,108079,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
4,4,25,108701,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
5,5,25,108786,1.386294,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
6,6,25,108797,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
7,7,25,108952,0.693147,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
8,8,25,111397,2.639057,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
9,9,25,114790,1.386294,True,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400


In [5]:
%time df_all.describe(include='all')

CPU times: user 1min 20s, sys: 30.3 s, total: 1min 50s
Wall time: 1min 50s


,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
count,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08,1.254970e+08,125497040,125497040,125497040,125497040,125497040,125497040,1.254970e+08
unique,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,2,2,NaN
top,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,NaN
freq,NaN,NaN,NaN,NaN,96028767,NaN,NaN,NaN,NaN,NaN,NaN,121312766,121474228,124124094,124369981,125164939,125489964,NaN
mean,6.274852e+07,2.746458e+01,9.727692e+05,1.750860e+00,NaN,2.015223e+03,6.334971e+00,2.571433e+01,1.560188e+01,3.041554e+00,1.772760e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.442335e+09
std,3.622788e+07,1.633051e+01,5.205336e+05,8.815504e-01,NaN,1.299140e+00,3.392866e+00,1.484178e+01,8.816411e+00,2.018242e+00,1.036237e+02,NaN,NaN,NaN,NaN,NaN,NaN,4.014999e+07
min,0.000000e+00,1.000000e+00,9.699500e+04,0.000000e+00,NaN,2.013000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,1.356998e+09
25%,3.137426e+07,1.200000e+01,5.223830e+05,1.098612e+00,NaN,2.014000e+03,3.000000e+00,1.300000e+01,8.000000e+00,1.000000e+00,8.800000e+01,NaN,NaN,NaN,NaN,NaN,NaN,1.411171e+09
50%,6.274852e+07,2.800000e+01,9.595000e+05,1.609438e+00,NaN,2.015000e+03,6.000000e+00,2.500000e+01,1.500000e+01,3.000000e+00,1.750000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.448064e+09
75%,9.412278e+07,4.300000e+01,1.354380e+06,2.302585e+00,NaN,2.016000e+03,9.000000e+00,3.800000e+01,2.300000e+01,5.000000e+00,2.640000e+02,NaN,NaN,NaN,NaN,NaN,NaN,1.476662e+09
